### Library

In [4]:
from datetime import datetime, timedelta
import numpy as np
import oandapy
import sys
sys.path.append('/Users/toshio/project/fx')
from config import *
from lib.inference import Inference
import time

In [5]:
def get_current_state():
    try:
        response = oanda.get_position(account_id, instrument ='USD_JPY')
        side = response.get('side')
        units = response.get('units')
    except oandapy.OandaError:
        side = 'zero_position'
        units = 0
    return side, units

In [6]:
def order(side, units):
    trade_expire = datetime.utcnow() + timedelta(minutes=1)
    trade_expire = trade_expire.isoformat("T") + "Z"
        
    response = oanda.create_order(
        acc_id,
        instrument='USD_JPY',
        units=units,
        side=side,
        type='market',
        expiry=trade_expire
    )

In [9]:
def transaction(units):
    #未決済取引情報を取得
    open_trades = oanda.get_trades(account_id)
    if len(open_trades['trades']) != 0:
        #未決済取引があればポジションをcloseする
        trade_id = open_trades['trades'][0]['id']
        closed_trades = oanda.close_trade(account_id, trade_id)
        profit = closed_trades['profit']
        print('profit:{}'.format(profit))
    else:
        pass

    #1時間後の差額を推定する
    inf = Inference()
    pred_delta=float(inf.pred[0][0])
    print('1時間後の推定差額は{}'.format(np.round(pred_delta, 4)))

    if pred_delta >= 0.1:
        print('Buy!')
        order('buy', units)
    elif pred_delta <= -0.1:
        print('Sell!')
        order('sell', units)
    elif -0.1 < pred_delta < 0.1:
        print('do nothing')

In [11]:
oanda = oandapy.API(environment="practice", access_token=token)
trading_unit = 1000

#100時間取引する
j = 0
while j < 1000:
    j +=1
    now = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    print('------transaction_{0:03}------'.format(j))
    print(now)
    transaction(trading_unit)
    print('-------------------------------------')
    time.sleep(5) #60秒に設定中

------transaction_001------
2018-09-17 00:07:53
profit:-1.2
1時間後の推定差額は0.0133
do nothing
-------------------------------------
------transaction_002------
2018-09-17 00:08:01
1時間後の推定差額は0.0103
do nothing
-------------------------------------
------transaction_003------
2018-09-17 00:08:08
1時間後の推定差額は-0.0026
do nothing
-------------------------------------
------transaction_004------
2018-09-17 00:08:15


KeyboardInterrupt: 